In [4]:
import numpy as np
from PIL import Image
import base64
import struct
import heapq
import io

### Compression utilities
def delta_encode(data):
    delta = [data[0]]
    for i in range(1, len(data)):
        delta.append(np.uint8(data[i] - data[i - 1]))
    return delta

def rle_encode(data):
    encoded = []
    count = 1
    for i in range(1, len(data)):
        if data[i] == data[i-1] and count < 255:
            count += 1
        else:
            encoded.extend([data[i-1], count])
            count = 1
    encoded.extend([data[-1], count])
    return encoded

def bitpack_4bit(data):
    packed = []
    for i in range(0, len(data), 2):
        first = data[i] & 0x0F
        second = data[i+1] & 0x0F if i+1 < len(data) else 0
        packed.append((first << 4) | second)
    return packed

def huffman_encode(data):
    freq = {}
    for byte in data:
        freq[byte] = freq.get(byte, 0) + 1
    heap = [[weight, [sym, ""]] for sym, weight in freq.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]: pair[1] = '0' + pair[1]
        for pair in hi[1:]: pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    huff_map = {sym: code for sym, code in heap[0][1:]}
    encoded_bits = ''.join([huff_map[b] for b in data])
    padded = encoded_bits + '0' * ((8 - len(encoded_bits) % 8) % 8)
    return bytes(int(padded[i:i+8], 2) for i in range(0, len(padded), 8))

def lzw_encode(data):
    dictionary = {bytes([i]): i for i in range(256)}
    w = b""
    codes = []
    for byte in data:
        wc = w + bytes([byte])
        if wc in dictionary:
            w = wc
        else:
            codes.append(dictionary[w])
            dictionary[wc] = len(dictionary)
            w = bytes([byte])
    if w:
        codes.append(dictionary[w])
    encoded_bytes = b''.join(struct.pack('>H', code) for code in codes)
    return encoded_bytes

### 🔧 Grayscale Encoder Pipeline
def encode_grayscale_lix(image_path):
    img = Image.open(image_path).convert("L").resize((128, 128))
    data = np.array(img).flatten()
    width, height = img.size

    methods = {}
    
    # Method 0: Delta only
    delta = delta_encode(data)
    methods[0] = bytes(delta)

    # Method 1: Delta + RLE
    delta_rle = rle_encode(delta)
    methods[1] = bytes(delta_rle)

    # Method 2: Bit-pack + Huffman (only if range fits 0-15)
    if np.all(data < 16):
        bitpacked = bitpack_4bit(data)
        huff = huffman_encode(bitpacked)
        methods[2] = huff

    # Method 3: Delta + LZW
    delta_lzw = lzw_encode(delta)
    methods[3] = delta_lzw

    # Choose smallest
    best_method = min(methods, key=lambda k: len(methods[k]))
    compressed = methods[best_method]

    # Construct .lix binary
    header = b'LIXG'
    meta = struct.pack('>HHB', width, height, best_method)
    return header + meta + compressed

### Save the .lix file
def save_lix_file(image_path, output_path):
    compressed_data = encode_grayscale_lix(image_path)
    with open(output_path, 'wb') as f:
        f.write(compressed_data)
    print(f"✅ Saved .lix file as {output_path}, size: {len(compressed_data)} bytes")

# Example usage:
save_lix_file("exp.jpg", "image_grayscale.lix")


✅ Saved .lix file as image_grayscale.lix, size: 13363 bytes


C:\Users\ELIXPO\AppData\Local\Temp\ipykernel_23268\3730197488.py:12: RuntimeWarning: overflow encountered in scalar subtract
  delta.append(np.uint8(data[i] - data[i - 1]))


In [5]:
import numpy as np
import struct
from PIL import Image

### Decompression utilities
def delta_decode(data):
    decoded = [data[0]]
    for i in range(1, len(data)):
        decoded.append((decoded[-1] + data[i]) % 256)
    return decoded

def rle_decode(data):
    decoded = []
    for i in range(0, len(data), 2):
        value = data[i]
        count = data[i + 1]
        decoded.extend([value] * count)
    return decoded

def bitunpack_4bit(data):
    unpacked = []
    for byte in data:
        unpacked.append((byte >> 4) & 0x0F)
        unpacked.append(byte & 0x0F)
    return unpacked

def huffman_decode(encoded_data, huff_map):
    reverse_map = {v: k for k, v in huff_map.items()}
    bit_string = ''.join(f'{byte:08b}' for byte in encoded_data)
    decoded = []
    buffer = ""
    for bit in bit_string:
        buffer += bit
        if buffer in reverse_map:
            decoded.append(reverse_map[buffer])
            buffer = ""
    return decoded

def lzw_decode(encoded_data):
    codes = [struct.unpack('>H', encoded_data[i:i + 2])[0] for i in range(0, len(encoded_data), 2)]
    dictionary = {i: bytes([i]) for i in range(256)}
    w = bytes([codes[0]])
    decoded = [w]
    for code in codes[1:]:
        if code in dictionary:
            entry = dictionary[code]
        elif code == len(dictionary):
            entry = w + w[:1]
        else:
            raise ValueError("Invalid LZW code")
        decoded.append(entry)
        dictionary[len(dictionary)] = w + entry[:1]
        w = entry
    return b''.join(decoded)

### Grayscale Decoder Pipeline
def decode_grayscale_lix(lix_path):
    with open(lix_path, 'rb') as f:
        data = f.read()

    # Parse header and metadata
    header = data[:4]
    if header != b'LIXG':
        raise ValueError("Invalid .lix file format")
    
    width, height, method = struct.unpack('>HHB', data[4:9])
    compressed_data = data[9:]

    # Decompress based on method
    if method == 0:  # Delta only
        decompressed = delta_decode(compressed_data)
    elif method == 1:  # Delta + RLE
        delta_rle = rle_decode(compressed_data)
        decompressed = delta_decode(delta_rle)
    elif method == 2:  # Bit-pack + Huffman
        # Note: Huffman decoding requires the Huffman map, which is not stored in the .lix file.
        # This implementation assumes the map is predefined or stored elsewhere.
        raise NotImplementedError("Huffman decoding requires the Huffman map")
    elif method == 3:  # Delta + LZW
        delta_lzw = lzw_decode(compressed_data)
        decompressed = delta_decode(delta_lzw)
    else:
        raise ValueError("Unknown compression method")

    # Reshape and return as image
    decompressed = np.array(decompressed, dtype=np.uint8).reshape((height, width))
    return Image.fromarray(decompressed, mode="L")

### Save the decoded image
def save_decoded_image(lix_path, output_path):
    img = decode_grayscale_lix(lix_path)
    img.save(output_path)
    print(f"✅ Decoded image saved as {output_path}")

# Example usage:
save_decoded_image("image_grayscale.lix", "decoded_image.jpg")

✅ Decoded image saved as decoded_image.jpg


In [3]:
import os

def compare_file_sizes(original_image_path, lix_file_path, decoded_image_path):
    # Get file sizes
    original_size = os.path.getsize(original_image_path)
    lix_size = os.path.getsize(lix_file_path)
    decoded_size = os.path.getsize(decoded_image_path)

    # Print the sizes
    print(f"Original Image Size: {original_size} bytes")
    print(f".lix File Size: {lix_size} bytes")
    print(f"Decoded Image Size: {decoded_size} bytes")

    # Compare sizes
    compression_ratio = original_size / lix_size if lix_size > 0 else float('inf')
    print(f"Compression Ratio (Original / .lix): {compression_ratio:.2f}")

# Example usage:
compare_file_sizes("exp2.jpg", "image_grayscale.lix", "decoded_image.jpg")

Original Image Size: 3468 bytes
.lix File Size: 11973 bytes
Decoded Image Size: 2624 bytes
Compression Ratio (Original / .lix): 0.29
